In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cargar el archivo Excel
file_path = '/content/drive/MyDrive/Fundamentos/Reporte_para_H2O_v2.xlsx'  # Cambia esto por la ruta de tu archivo
df = pd.read_excel(file_path)

# Ver las primeras filas del DataFrame para entender su estructura
print(df.head())

  Fecha de venta Canal de Venta                      Cliente  \
0     2024-01-30  Mercado Libre     AILENGUADALUPEMARTORELLA   
1     2024-01-30  Mercado Libre  ROSSOCRISTIAN20231018184201   
2     2024-01-30  Mercado Libre                   SILVYN2011   
3     2024-01-30  Mercado Libre                        J4NU$   
4     2024-01-30  Mercado Shops         ALEXISMARILINREINERT   

                      Provincia  Cantidad vendida  Precio unitario  \
0                       Neuquén                 6           4750.0   
1  CIUDAD AUTONOMA BUENOS AIRES                 8           5389.0   
2                  Buenos Aires                 1           4750.0   
3               Capital Federal                10           4750.0   
4                      Misiones                 1           2690.0   

   Precio unitario dolar  Unnamed: 7  Total venta dolar  Valor MEP  \
0               4.088624         NaN          24.531745    1161.76   
1               4.638652         NaN          37.10921

In [4]:
# Convertir "fecha de venta" en un formato de fecha
df['Fecha de venta'] = pd.to_datetime(df['Fecha de venta'])

# Crear una columna para el mes y el año
df['dia_mes_año'] = df['Fecha de venta'].dt.to_period('D')

# Agrupar las ventas por "número de publicación" y "mes_año", sumando "cantidad vendida"
df_ventas_diarias = df.groupby(['Número de publicación', 'dia_mes_año'])['Cantidad vendida'].sum().reset_index()

# Renombrar la columna de cantidad vendida
df_ventas_diarias.rename(columns={'Cantidad vendida': 'Unidades vendidas por dia'}, inplace=True)

# Crear un DataFrame con todos los meses y publicaciones posibles para asegurarse de tener meses con 0 ventas
rango_dias = pd.period_range(df['Fecha de venta'].min(), df['Fecha de venta'].max(), freq='D')
todos_los_dias_publicaciones = pd.MultiIndex.from_product(
    [df['Número de publicación'].unique(), rango_dias],
    names=['Número de publicación', 'dia_mes_año']
)

# Hacer un merge (outer join) con el DataFrame original para rellenar los meses sin ventas con 0
df_completo = pd.DataFrame(index=todos_los_dias_publicaciones).reset_index()
df_completo = pd.merge(df_completo, df_ventas_diarias, on=['Número de publicación', 'dia_mes_año'], how='left').fillna(0)

# Convertir el periodo de nuevo en string (opcional, si prefieres que se vea como "01-2022")
df_completo['dia_mes_año'] = df_completo['dia_mes_año'].astype(str)

print(df_completo)


       Número de publicación dia_mes_año  Unidades vendidas por dia
0              MLA1401793983  2022-05-30                        0.0
1              MLA1401793983  2022-05-31                        0.0
2              MLA1401793983  2022-06-01                        0.0
3              MLA1401793983  2022-06-02                        0.0
4              MLA1401793983  2022-06-03                        0.0
...                      ...         ...                        ...
134967          MLA885339643  2024-08-25                        0.0
134968          MLA885339643  2024-08-26                        0.0
134969          MLA885339643  2024-08-27                        0.0
134970          MLA885339643  2024-08-28                        0.0
134971          MLA885339643  2024-08-29                        0.0

[134972 rows x 3 columns]


In [5]:
# Guardar el df_unificado a un archivo de excel, para poder analizarlo
df_completo.to_excel('/content/drive/MyDrive/Fundamentos/df_unidades_diarias.xlsx', index=False)

In [6]:
# Pivotar el DataFrame para tener los "número de publicación" como columnas y "mes_año" como índice
df_pivot = df_completo.pivot(index='dia_mes_año', columns='Número de publicación', values='Unidades vendidas por dia')

# Asegurarse de que el índice esté en el formato datetime si no lo está ya
df_pivot.index = pd.to_datetime(df_pivot.index, format='%Y-%m-%d')

print(df_pivot)

Número de publicación    0  MLA1103986934  MLA1104082705  MLA1104097617  \
dia_mes_año                                                               
2022-05-30             0.0            0.0            0.0            0.0   
2022-05-31             0.0            0.0            0.0            0.0   
2022-06-01             0.0            0.0            0.0            0.0   
2022-06-02             0.0            0.0            0.0            0.0   
2022-06-03             0.0            0.0            0.0            0.0   
...                    ...            ...            ...            ...   
2024-08-25             0.0            0.0            0.0            0.0   
2024-08-26             0.0            0.0            0.0            0.0   
2024-08-27             0.0            0.0            0.0            0.0   
2024-08-28             0.0            0.0            0.0            0.0   
2024-08-29             0.0            0.0            0.0            0.0   

Número de publicación  M

In [7]:
# Guardar el df_unificado a un archivo de excel, para poder analizarlo
df_pivot.to_excel('/content/drive/MyDrive/Fundamentos/df_pivot_diario.xlsx', index=True)